In [4]:
import pandas as pd
import numpy as np 
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import re
from collections import Counter

import spacy
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv('/Users/pranjalmishra/Desktop/python/ipc_sections.csv', encoding='unicode_escape')

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

def spacy_preprocessing(text):
    doc = nlp(text)
 
    lemmatized_text = ' '.join([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
   
    tokens = [token for token in lemmatized_text.split() if token.lower() not in STOPWORDS]

    entities = [ent.text for ent in doc.ents]
    non_entity_tokens = [token for token in tokens if token not in entities]

    preprocessed_text = ' '.join(non_entity_tokens)

    return preprocessed_text

def preprocessingDesc(text):
 
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)  
    text = text.lower()  
    
   
    return spacy_preprocessing(text)

df['Description'] = df['Description'].apply(preprocessingDesc)

user_question = input("Please enter your text: ")
print("You entered:", user_question)
preprocessed_question = preprocessingDesc(user_question)
preprocessed_descriptions = [preprocessingDesc(desc) for desc in df['Description'].tolist()]

vectorizer = TfidfVectorizer()
description_vectors = vectorizer.fit_transform(preprocessed_descriptions)
question_vector = vectorizer.transform([preprocessed_question])

similarities = cosine_similarity(question_vector, description_vectors)
similar_indices = np.where(similarities[0] >= 0.1)[0]  
recommended_ipcs = df.loc[similar_indices, 'Section']
print("Recommended IPC(s):", ', '.join(map(str, recommended_ipcs)))


You entered: intentionally adulterating a drug or medical preparation to reduce its effectiveness or make it harmful, intending to sell it for medicinal purposes
Recommended IPC(s): IPC_233, IPC_234, IPC_257, IPC_258, IPC_272, IPC_274, IPC_275, IPC_276, IPC_328, IPC_351, IPC_372, IPC_430, IPC_489D


In [1]:
import numpy as np 
import pandas as pd

import nltk
import torch
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import re
from collections import Counter
from gensim.parsing.preprocessing import remove_stopwords

nltk.download('punkt')
nltk.download('stopwords')


nlp = spacy.load('en_core_web_sm')

df = pd.read_csv('/Users/pranjalmishra/Desktop/python/ipc_sections.csv', encoding='unicode_escape')

def preprocessingDesc(text):

    text = re.sub(r'[^\w\s]', '', text.lower())

    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]

    return ' '.join(tokens)

def preprocess_text(text):
 
    doc = nlp(text.lower())
    filtered_tokens = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV'] and not token.is_stop]
    return ' '.join(filtered_tokens)

df['Processed_Description'] = df['Description'].apply(preprocessingDesc)


user_question = input("Please enter your text: ")
processed_question = preprocess_text(user_question)


preprocessed_descriptions = [preprocess_text(desc) for desc in df['Processed_Description']]

vectorizer = TfidfVectorizer()
description_vectors = vectorizer.fit_transform(preprocessed_descriptions)
question_vector = vectorizer.transform([processed_question])

similarities = cosine_similarity(question_vector, description_vectors)

similarity_threshold = 0.1

similar_indices = np.where(similarities[0] >= similarity_threshold)[0]


recommended_ipcs = df.loc[similar_indices, ['Section', 'Punishment']]
print("You entered:", user_question)
print("Recommended IPC(s):", ', '.join(map(str, recommended_ipcs)))

for index, row in recommended_ipcs.iterrows():
    print(f"IPC Section: {row['Section']}, Punishment: {row['Punishment']}")



/var/folders/dn/t8s0kyv52pbfv6q_x3r132rw0000gn/T/ipykernel_21637/717149160.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


You entered: I, Amit Kumar, was returning to my home on a bicycle when I met Lokesh, who was drunk, on the way. He abused and attacked me for some old thing, which caused serious injuries to my eyes and nose and on the way he also threatened to kill me
Recommended IPC(s): Section, Punishment
IPC Section: IPC_166, Punishment: Simple Imprisonment for 1 Year or Fine or Both
IPC Section: IPC_152, Punishment: 3 Years or Fine or Both
IPC Section: IPC_195A, Punishment: 7 Years or Fine or Both
IPC Section: IPC_229, Punishment: 2 Years or Fine or Both
IPC Section: IPC_283, Punishment: Fine
IPC Section: IPC_300, Punishment: nan
IPC Section: IPC_351, Punishment: nan
IPC Section: IPC_428, Punishment: 2 Years or Fine or Both
